# Step 1: Import packages and define functions

In [20]:
# start with loading the required packages
import os
import ntpath
import datetime
import pandas as pd
from Bio import SeqIO
from scipy import signal
from iupred2a import iupred2a
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from tqdm.auto import tqdm
from iupred2a import iupred2a
import bz2
import pickle
import _pickle as cPickle

from multiprocessing import Pool
# Windows
CWD = os.getcwd()

In [ ]:
# To load compressed pickle file
def decompress_pickle(file):
 data = bz2.BZ2File(file, 'rb')
 data = cPickle.load(data)
 return data

In [ ]:
# Pickle a file and compress it into a file (to output data if required)
def compressed_pickle(title, data):
 with bz2.BZ2File(title + '.pbz2', 'w') as f:cPickle.dump(data, f)
# will be done using
# compressed_pickle('output_name', data)

In [21]:
# define amino acids and their hydrophobicity index
RESIDUES = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

# Kyte & Doolittle {kd} index of hydrophobicity
HP = {'A': 1.8, 'R':-4.5, 'N':-3.5, 'D':-3.5, 'C': 2.5,
      'Q':-3.5, 'E':-3.5, 'G':-0.4, 'H':-3.2, 'I': 4.5,
      'L': 3.8, 'K':-3.9, 'M': 1.9, 'F': 2.8, 'P':-1.6,
      'S':-0.8, 'T':-0.7, 'W':-0.9, 'Y':-1.3, 'V': 4.2, 'U': 0.0}

In [22]:
# define classes and functions
import time
class IuPred:
    def __init__(self, glob, short, long):
        self.glob = glob
        self.short = short
        self.long = long


class HydroPhobicIndex:
    def __init__(self, hpilist):
        self.hpilist = hpilist
        
def calc_idrpred(args):
    idx, seq = args
    glob = iupred2a.iupred(str(seq), 'glob')
    short = iupred2a.iupred(str(seq), 'short')
    long = iupred2a.iupred(str(seq), 'long')
    idrpred = IuPred(glob, short, long)
    return idx, idrpred
        

class MakeMatrix:
    def __init__(self, dbfasta):  
        self.df = pd.DataFrame()
        executables = [
             'self.fasta2df(dbfasta)',
             'self.amino_acid_analysis()',
             'self.idr_iupred()',
             'self.hydrophobic()',
             'self.add_iupred_features()',
             'self.add_hydrophobic_features()',
             'self.add_biochemical_combinations()',
             'self.add_lowcomplexity_features()' ,
#             #'self.add_plaac()'
        ]        
        for e in executables:
            start = time.time()     
            print(e)
            exec(e)
            end = time.time()        
            print(str(round(end - start, 2))+'s '+e)

    def fasta2df(self, dbfasta):
        rows = list()
        with open(dbfasta) as f:
            for record in SeqIO.parse(dbfasta, 'fasta'):
                seqdict = dict()
                seq = str(record.seq)
                id = record.description.split('|')                
                if id[0] == 'sp':
                    uniprot_id = id[1]
                    name = id[2].split(' ')[0]
                    rows.append([name, uniprot_id, seq])
                elif id[0] == 'tr':
                    uniprot_id = id[1]
                    name = id[2].split(' ')[0]
                    rows.append([name, uniprot_id, seq])
                else:
                    uniprot_id = id[0]
                    name = id[2].split(' ')[0]
                    rows.append([name, uniprot_id, seq])                    
        self.df = pd.DataFrame(rows, columns=['protein_name', 'uniprot_id', 'sequence'])
        print(len(self.df))

    def idr_iupred(self):
        self.df['iupred'] = object
        data = list(self.df["sequence"].iteritems())
        p = Pool(32)
        for idx, idrpred in tqdm(p.imap(calc_idrpred, data), total=self.df.shape[0]):
            self.df.at[idx, 'iupred'] = idrpred
            
    def hydrophobic(self):
        for index, row in self.df.iterrows():
            hpilst = pd.Series(list(row['sequence'])).map(HP).tolist()
            self.df.loc[index, 'HydroPhobicIndex'] = HydroPhobicIndex(hpilst)
            
    def amino_acid_analysis(self):
        for res in RESIDUES:
            self.df['fraction_'+res] = self.df['sequence'].str.count(res) / self.df['sequence'].str.len()
        self.df['length'] = self.df['sequence'].str.len()
        for index, row in tqdm(self.df.iterrows(), total=self.df.shape[0]):
        #for index, row in self.df.iterrows():
            seq = row['sequence']   
            seqanalysis = ProteinAnalysis(seq)
            acidist = seqanalysis.get_amino_acids_percent() 
            self.df.loc[index, 'IEP'] = seqanalysis.isoelectric_point()
            if 'X' not in seq and 'B' not in seq:
                self.df.loc[index, 'molecular_weight'] = seqanalysis.molecular_weight()
            if 'U' not in seq and 'X' not in seq and 'B' not in seq:
                self.df.loc[index, 'gravy'] = seqanalysis.gravy()
          

    def add_iupred_features(self):
        for index, row in tqdm(self.df.iterrows(), total=self.df.shape[0]):
        #for index, row in self.df.iterrows():
            idr = row['iupred'].glob[0]
            self.df.loc[index, 'idr_percetage'] = sum(i > .5 for i in list(idr))
            self.df.loc[index, 'idr_50'] = sum(i > .5 for i in list(idr)) / len(str(row['sequence']))
            self.df.loc[index, 'idr_60'] = sum(i > .6 for i in list(idr)) / len(str(row['sequence']))
            self.df.loc[index, 'idr_70'] = sum(i > .7 for i in list(idr)) / len(str(row['sequence']))
            self.df.loc[index, 'idr_80'] = sum(i > .8 for i in list(idr)) / len(str(row['sequence']))
            self.df.loc[index, 'idr_90'] = sum(i > .9 for i in list(idr)) / len(str(row['sequence']))

    def add_hydrophobic_features(self):
        hpi0, hpi1, hpi2, hpi3, hpi4, hpi5 = list(), list(), list(), list(), list(), list() 
        for index, row in tqdm(self.df.iterrows(), total=self.df.shape[0]):
        #for index, row in self.df.iterrows():
            sw = convolve_signal(row['HydroPhobicIndex'].hpilist, window=30)
            hpi0.append(sum(i < -1.5 for i in sw) / len(sw))
            # self.df.loc[index, 'hpi_<-1.5_frac'] = hpi
            hpi1.append(sum(i < -2.0 for i in sw) / len(sw))
            # self.df.loc[index, 'hpi_<-2.0_frac'] = hpi
            hpi2.append(sum(i < -2.5 for i in sw) / len(sw))
            # self.df.loc[index, 'hpi_<-2.5_frac'] = hpi
            hpi3.append(sum(i < -1.5 for i in sw))
            # self.df.loc[index, 'hpi_<-1.5'] = hpi
            hpi4.append( sum(i < -2.0 for i in sw))
            # self.df.loc[index, 'hpi_<-2.0'] = hpi
            hpi5.append(sum(i < -2.5 for i in sw))
            # self.df.loc[index, 'hpi_<-2.5'] = hpi 
        self.df['hpi_<-1.5_frac'] = hpi0
        self.df['hpi_<-2.0_frac'] = hpi1
        self.df['hpi_<-2.5_frac'] = hpi2
        self.df['hpi_<-1.5'] = hpi3
        self.df['hpi_<-2.0'] = hpi4
        self.df['hpi_<-2.5'] = hpi5
            

    def add_biochemical_combinations(self):
        df = self.df
        df = df.assign(Asx=df['fraction_D'] + df['fraction_N'])
        df = df.assign(Glx=df['fraction_E'] + df['fraction_Q'])
        df = df.assign(Xle=df['fraction_I'] + df['fraction_L'])
        df = df.assign(Pos_charge=df['fraction_K'] + df['fraction_R'] + df['fraction_H'])
        df = df.assign(Neg_charge=df['fraction_D'] + df['fraction_E'])
        df = df.assign(Aromatic=df['fraction_F'] + df['fraction_W'] + df['fraction_Y'] + df['fraction_H'])
        df = df.assign(Alipatic=df['fraction_V'] + df['fraction_I'] + df['fraction_L'] + df['fraction_M'])
        df = df.assign(Small=df['fraction_P'] + df['fraction_G'] + df['fraction_A'] + df['fraction_S'])
        df = df.assign(Hydrophilic=(df['fraction_S'] + df['fraction_T'] + df['fraction_H'] + 
                                    df['fraction_N'] + df['fraction_Q'] + df['fraction_E'] +
                                    df['fraction_D'] + df['fraction_K'] + df['fraction_R']))
        df = df.assign(Hydrophobic= (df['fraction_V'] + df['fraction_I'] + df['fraction_L'] +
                                     df['fraction_F'] + df['fraction_W'] + df['fraction_Y'] +
                                     df['fraction_M']))
        
        # Added in version 2
        for dimer in ['GV', 'VG', 'VP', 'PG', 'FG', 'RG', 'GR', 'GG', 'YG', 'GS', 'SG', 'GA', 'GF', 'GD', 'DS']:
            self.df[dimer] = self.df['sequence'].str.count(dimer)
        df = df.assign(alpha_helix=df['fraction_V'] + df['fraction_I'] + df['fraction_Y'] + df['fraction_F']
                      + df['fraction_W'] + df['fraction_L'])
        df = df.assign(beta_turn=df['fraction_N'] + df['fraction_P'] + df['fraction_G'] + df['fraction_S'])
        df = df.assign(beta_sheet=df['fraction_E'] + df['fraction_M'] + df['fraction_A'] + df['fraction_L'])
        # Calculates the aromaticity value of a protein according to Lobry, 1994. 
        # It is simply the relative frequency of Phe+Trp+Tyr.
        df = df.assign(aromaticity=df['fraction_F'] + df['fraction_W'] + df['fraction_Y'])

        
        
        
        self.df = df
        del df
        
    def add_lowcomplexityscore(self):
        lcs_window = 20
        lcs_cutoff = 7
        for index, row in self.df.iterrows():
            seq = str(row['sequence'])
            if len(seq) > lcs_window+1:
                sig = list()
                for i in range(len(seq)):
                    window = (seq[i: i+lcs_window])
                    if len(window) == lcs_window:
                        acid_comp = len(list(set(window)))
                        sig.append(acid_comp)
                score = sum([1 if i <= 7 else 0 for i in sig])
                self.df.loc[index, 'lcs_score'] = score
                self.df.loc[index, 'lcs_fraction'] = score / len(sig)
                
                
    def add_lowcomplexity_features(self):
        n_window = 20
        cutoff = 7       
        n_halfwindow = int(n_window / 2)        
        lcs_lowest_complexity = list()
        lcs_scores = list()
        lcs_fractions = list()
        for index, row in tqdm(self.df.iterrows(), total=self.df.shape[0]):      
            # Determine low complexity scores
            seq = str(row['sequence'])
            lcs_acids = list()
            sig = list()
            
            # New
            lc_bool = [False] * len(seq)
            for i in range(len(seq)):
                if i < n_halfwindow:
                    peptide = seq[:n_window]        
                elif i+n_halfwindow > int(len(seq)):
                    peptide = seq[-n_window:]        
                else:
                    peptide = seq[i-n_halfwindow:i+n_halfwindow]       
                complexity = (len(set(peptide)))
                if complexity <= 7:
                    for bool_index in (i-n_halfwindow, i+n_halfwindow):
                        try:
                            lc_bool[bool_index] = True
                        except IndexError:
                            pass
                    lcs_acids.append(seq[i])
                sig.append(complexity)            
            # Adding low complexity scores to list
            low_complexity_list = pd.DataFrame({'bool':lc_bool, 'acid':list(seq)}, index=None)
            lcs_lowest_complexity.append(min(sig))
            lcs_scores.append(len(low_complexity_list.loc[low_complexity_list['bool'] == True]))
            lcs_fractions.append(len(low_complexity_list.loc[low_complexity_list['bool'] == True]) / len(seq))
            low_complexity_list = pd.DataFrame({'bool':lc_bool, 'acid':list(seq)}, index=None)
            if len(lcs_acids) >= n_window:
                for i in RESIDUES:
                    self.df.loc[index ,i+'_lcscore'] = (len(low_complexity_list.loc[
                        (low_complexity_list['bool'] == True) &
                        (low_complexity_list['acid'] == i)])
                    )
                    self.df.loc[index ,i+'_lcfraction'] = (len(low_complexity_list.loc[
                        (low_complexity_list['bool'] == True) & 
                        (low_complexity_list['acid'] == i)]) / len(lcs_acids)
                    )
        self.df['lcs_fractions'] = lcs_fractions
        self.df['lcs_scores'] = lcs_scores
        self.df['lcs_lowest_complexity'] = lcs_lowest_complexity
        
def convolve_signal(sig, window=25):
    win = signal.hann(window)
    sig = signal.convolve(sig, win, mode='same') / sum(win)
    return sig


def average(l):
    return sum(l) / len(l)

In [23]:
def main(name, fasta_path, operating_system='Windows'):
    # Change pathing
    """ Generates and saves a file which contains features of a protein sequence in the defined output directory.
    Parameters:
        name: Name of the file.
        fasta_path: Path of the fasta file which needs to be featured.
        operating_system: String which indicates which operating system is used only 'Windows' available.
    """
    data = MakeMatrix(fasta_path)   
    #now = datetime.datetime.now()
    #date = (str(now.day) + '-' + str(now.month)  + '-' +  str(now.year))
    #define output directory for .pkl file here
    #data.df.to_pickle('/output/path/'+ name + '_small_' + date + '.pkl')
    #print('Generated file: ' + name + '_llps_f2f_' + date + '.pkl')
    return data.df

In [ ]:
def set_identifiers(df, uniprot_ids, identifier_name):
    uniprot_ids = [s.strip() for s in uniprot_ids.splitlines()]
    df[identifier_name] = 0
    for prot_id in uniprot_ids:
        df.loc[df['uniprot_id'] == prot_id, identifier_name] = 1
        if (len(df.loc[df['uniprot_id'] == prot_id])) == 0:
            print(prot_id+' is not found.')
    return df

In [ ]:
# load required packages
# coding: utf-8
%matplotlib inline
import os
import sys
import warnings
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import sample
import collections
from statistics import mean
from tqdm import tqdm
import pickle5 as pickle

# Preprocessing
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
# Sklearn imports
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import *

from sklearn.feature_selection import VarianceThreshold

# Different machine learning models from sklearn
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import average_precision_score

CWD = os.getcwd()
RANDOMSTATE = 42

class IuPred:
    '''
    Needs description.
    '''    
    def __init__(self, glob, short, long):
        self.glob = glob
        self.short = short
        self.long = long


class HydroPhobicIndex:
    '''
    Needs description.
    '''
    def __init__(self, hpilist):
        self.hpilist = hpilist

In [ ]:
# define functions required to train the machine learning classifier
def preprocess_data(df, scaler):
    info = df.select_dtypes(include=['object'])
    y = df[instance]
    X = df.drop([instance], axis=1)
    X = X._get_numeric_data()
    columns = X.columns
    
    X = scaler.fit_transform(X)    
    X = pd.DataFrame(X, columns=columns)
    X[instance] = y
    X = X.merge(info, how='outer', left_index=True, right_index=True)    
    return X


def remove_correlating_features(df, cutoff=.95, plot=False):
    # Set matplotlib parameters
    sns.set(rc={'figure.figsize':(8,6)})
    plt.rcParams['pdf.fonttype'] = 42
    plt.rcParams['ps.fonttype'] = 42
    
    print('Correlation: Features before %s' % df.shape[1])
    
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > cutoff)]
    # Drop features
    df = df.drop(to_drop, axis=1)
    
    
    print('Correlation: Features after %s' % df.shape[1])
    if plot:
        # Plot heatmap
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.heatmap(data=df.corr(), cmap=sns.color_palette("coolwarm", 20), vmin=-1, vmax=1)
        plt.savefig(os.getcwd()+ '\\data\graphs\\6.Heatmap_after_correlation.pdf', transparent=True)
        plt.show()
    return df


def varianceSelection(df, threashold=.8):
    if not isinstance(df, pd.core.frame.DataFrame):
        logger.error('[%s] : [ERROR] Variance selection only possible on Dataframe not %s',
                                     datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'), type(df))
        sys.exit(1)
    sel = VarianceThreshold(threshold=(threashold * (1 - threashold)))
    sel.fit_transform(df)
    return df[[c for (s, c) in zip(sel.get_support(), df.columns.values) if s]]


def filter_low_variance(df, non_feature_columns, variance_cutoff=.8):
    df_variancetreshold = varianceSelection(df.drop(non_feature_columns, axis=1), variance_cutoff)
    for col in non_feature_columns:
        df_variancetreshold[col] = df[col]
    return df_variancetreshold


def remove_low_variance_features(df, variance_cutoff, plot=False):
    print('Low varience: Features before %s' % df.shape[1])
    non_feature_columns = ['protein_name', 'sequence', 'llps']
    df_variance = filter_low_variance(df, non_feature_columns, variance_cutoff)
    print('Low varience: Features after  %s' % df_variance.shape[1])
    
    # Set matplotlib parameters
    if plot:
        sns.set(rc={'figure.figsize':(8,6)})
        plt.rcParams['pdf.fonttype'] = 42
        plt.rcParams['ps.fonttype'] = 42

        # Plot heatmap
        sns.set(rc={'figure.figsize':(11.7,8.27)})
        sns.heatmap(data=df_variance.corr(), cmap=sns.color_palette("coolwarm", 20), vmin=-1, vmax=1)

        plt.savefig(os.getcwd()+ '\\data\graphs\\6.heatmap_after_low_variance.pdf', transparent=True)
        plt.show()
    return df_variance


def get_feature_importances(df, clf):
    '''
    Needs description.
    '''    
    fi = clf.feature_importances_
    fi = pd.DataFrame(fi).transpose()
    fi.columns = df.columns[1:]
    fi = fi.melt()
    fi = fi.sort_values('value', ascending=False)
    fi = fi.loc[fi['value'] > 0.0]
    return fi


def select_testing_data(df, instance, ratio=1, info_columns=['protein_name', 'sequence'], random=False):
    '''
    Class that equalizes the data based on the number of positives.
    :param df: dataframe with all the features and instance
    :param instance: string of instance columns
    :param ratio: ratio of how many negatives should be added (2 = twice as much negative)
    :param random: bool which gives a random test set with both instances randomly picked.
    :return: 'out' dataframe with new data, and 'df_info' dataframe with the information of 
    protein name and sequence.
    '''
    pos = df.loc[df[instance] == 1]
    neg_index = df.loc[df[instance] == 0].index
    negsubset_index = sample(set(neg_index), len(pos) * ratio)
    if random:
        possubset_index = sample(set(df.index), len(pos))
        negsubset_index = sample(set(df.index), len(pos))
        pos = df.loc[possubset_index]
        pos[instance] = pos[instance] = 1
    neg = df.loc[negsubset_index]
    out = pos.append(neg)
    out = out.reset_index(drop=True)
    df_info = out[info_columns]
    out = out.drop(labels=info_columns, axis=1)
    return out, df_info


def calculate_performance(clf, X_test, y_test):
    '''
    Needs description.
    '''    
    prediction = clf.predict(X_test)
    correctness = prediction == y_test
    distribution = correctness.value_counts(True)
    return distribution


def train_model(data, clf, instance):
    df, df_info = select_testing_data(data, instance, ratio=1, random=False)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(instance, axis=1),
                                                        df[instance],
                                                        test_size=0.5,
                                                        random_state=RANDOMSTATE,
                                                        )
    clf = clf.fit(X_train, y_train)
    return clf, X_test, y_test

    
def plot_pca(df, title='', analysis_name=''):
    # Set matplotlib parameters
    sns.set(rc={'figure.figsize':(8,6)})
    plt.rcParams['pdf.fonttype'] = 42
    plt.rcParams['ps.fonttype'] = 42
    
    
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(df.drop(['sequence', 'protein_name'],axis=1))
    pca_df = pd.DataFrame(X_pca, columns=['component1', 'component2'])
    pca_df['protein_name'] = df['protein_name']
    pca_df['llps'] = df.reset_index()['llps']


    sns.scatterplot(data=pca_df.loc[pca_df['llps'] == 0],
                    x = 'component1',
                    y = 'component2',
                    alpha = .5,
                    color = '#AEB6BF',
                    rasterized=True,
                   )
    sns.scatterplot(data=pca_df.loc[pca_df['llps'] == 1],
                    x = 'component1',
                    y = 'component2',
                    alpha = 1,
                    color = '#CD6155',                
                   )
    
    
    # print(pca_df.loc[(pca_df['llps'] == 1) & (pca_df['component1'] >= 650)])
    
    plt.ylabel('Component1 (' + str(round(pca.explained_variance_ratio_[0] * 100, 2)) + '%)')
    plt.xlabel('Component2 (' + str(round(pca.explained_variance_ratio_[1] * 100, 2)) + '%)')
    plt.title(title)
    plt.savefig(os.getcwd()+ f'\\analysis\\{analysis_name}\\PCA_plot.pdf', transparent=True)
    plt.show()


def average(l):
    return sum(l) / len(l)

In [ ]:
def get_test_train_indexes(data, label , ratio=1, randomized=False):
    '''
    Function: Will oversample the positive data with randomly selected negative samples.
    Returns: List with indexes which contain positive and negative samples.
    '''
    positive_instances = set(data.loc[data[label] == 1].index)
    negative_instances = set(data.loc[data[label] == 0].index)
    n_positives = data.loc[data[label] == 1].shape[0]    
    indexes = list()    
    while len(negative_instances) >= 1:    
        if len(negative_instances) > n_positives * ratio:
            sample_set = sample(negative_instances, (n_positives * ratio))
        else:
            sample_set = list(negative_instances)
            size = (len(sample_set))
            short = (len(positive_instances ) * ratio) - size        
            shortage = sample(set(data.loc[data[label] == 0].index), short)
            sample_set = (sample_set+ shortage)
        indexes.append((list(positive_instances) + list(sample_set)))
        negative_instances.difference_update(set(sample_set))
    return(indexes)


def evaluate_model(data, label, index_list, names, classifiers, testing_size, randomize=False):
    evaluations=dict()
    for name, clf in zip(names, classifiers):   
        scores=list()   
        non_numeric_columns = (list(data.select_dtypes(include='object')))
        data = data.select_dtypes([np.number])
        print(f'Now training: {name}')
        for index in tqdm(index_list):
            # Get index values.
            df = data.loc[index]
            
            # Randomize
            if randomize:
                pass
                
            # Specify data and labels.
            X = df.drop(instance, axis=1)
            y = df[label]

            # split data and train model.
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testing_size)
            clf.fit(X_train, y_train)

            # get predictions
            y_pred = clf.predict(X_test)
            y_proba = clf.predict_proba(X_test)
            scores.append([list(y_test), y_pred, y_proba])        
        evaluations[name] = scores        
    return evaluations


def get_model_evaluations(processed_data, ratio=1, testing_size=.1, label='llps'):
    names = [
        "Random Forest",
        "Gradient Boosting",
        "Extra Trees",
        "Decision Tree",
        "KNeighbors",
        "C-Support Vector",
        "Gaussian Process",
        "Neural Net",
        "AdaBoost",
        "Gaussian Naive Bayes",
    ]

    classifiers = [
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        ExtraTreesClassifier(),
        DecisionTreeClassifier(),
        KNeighborsClassifier(),
        SVC(probability=True),
        GaussianProcessClassifier(1.0 * RBF(1.0), max_iter_predict=100),
        MLPClassifier(),
        AdaBoostClassifier(),
        GaussianNB(),
    ]

    data = processed_data.copy()
    index_list = get_test_train_indexes(data, label, ratio=1)

    # evaluate models.
    evaluations = evaluate_model(data, label, index_list, names, classifiers, testing_size)
    return evaluations

# data preprocessing
def preprocess_and_scaledata(data, instance):
    try:
        data = data.drop(['iupred', 'HydroPhobicIndex', 'uniprot_id', 'PRDaa'], axis=1)
    except KeyError:
        data = data.drop(['iupred', 'HydroPhobicIndex', 'uniprot_id'], axis=1)
    data = data.fillna(value=0)
    print(data.shape)
    print('Number of phase separating proteins in dataset: '+str(data.loc[data[instance] == 1].shape[0]))
    print(data.shape)
    
    # scaler = QuantileTransformer(output_distribution='normal')
    scaler = QuantileTransformer()
    df = data.copy()
    processed_data = df.fillna(0)
    processed_data = preprocess_data(processed_data, scaler)
    #processed_data = remove_correlating_features(processed_data, cutoff=.95)
    #processed_data = remove_low_variance_features(processed_data, variance_cutoff=0.08)
    return processed_data

def split_dataframe_data(data, label, n=30, n_negative=30):
    positive_label_sample = data[data[label] == 1].sample(n)
    negative_label_sample = data[data[label] == 0].sample(n_negative)
    train_data = data.drop(list(positive_label_sample.index))
    train_data = train_data.drop(list(negative_label_sample.index))
    test_data = shuffle(positive_label_sample.append(negative_label_sample)).reset_index(drop=True)      
    return train_data, test_data,

In [ ]:
class EnsembleRandomForest:
    def __init__(self):
        self.estimators = list()

    def fit(self, data, index_list, label):
        estimators=list()   
        non_numeric_columns = (list(data.select_dtypes(include='object')))
        data = data.select_dtypes([np.number])
        for index in tqdm(index_list):
            df = data.loc[index]
            X = df.drop(instance, axis=1)
            y = df[label]
            clf = RandomForestClassifier()
            clf.fit(X, y)
            estimators.append(clf)
        self.estimators = estimators

    def predict_proba(self, df, label):
        probabilities = list()
        for clf in self.estimators:
            probability = clf.predict_proba(df)[:,1]
            probabilities.append(probability)
        probabilities = np.array(probabilities)
        predictions = list()
        for i in range(probabilities.shape[1]):
            prediction = np.average(probabilities[:,i])
            predictions.append(prediction)
        return predictions

In [ ]:
def predict_proteome(df, clf, instance, testing_size, feature_imp=True, remove_training=False, second_df=pd.DataFrame()):
    pd.set_option('mode.chained_assignment', None)    
    prediction = df.select_dtypes(include='object')
    df = df.select_dtypes([np.number])
    if len(second_df) > 0:
        prediction = second_df.select_dtypes(include='object')
        second_df = second_df.select_dtypes([np.number]) 
    indexes = get_test_train_indexes(df, instance)
    count = 0  
    fi_data = None
    for index in tqdm(indexes):
        df_fraction = df.loc[index]        
    # Also consider X_test index for prediction in the proteome    
        X = df_fraction.drop(instance, axis=1)        
        y = df_fraction[instance]
        clf.fit(X, y)           
    # Feature importance
        if feature_imp:        
            #X = df_fraction.drop('llps', axis=1)
            fi = clf.feature_importances_
            fi = pd.DataFrame(fi).transpose()
            fi.columns = X.columns
            fi = fi.melt()
            fi = fi.sort_values('value', ascending=False) 
            if fi_data is None:
                fi_data = fi            
            else:
                fi_data = pd.merge(fi_data, fi, on='variable')  
    # Make prediction
        if len(second_df) > 0:
            probability = clf.predict_proba(second_df.drop(instance, axis=1))[:, 1]
            prediction['probability_'+str(count)] = probability
        else:
            probability = clf.predict_proba(df.drop(instance, axis=1))[:, 1]  
            prediction['probability_'+str(count)] = probability        
    # Removing prediction that were used in the train test set.
        if remove_training:
            for i in index:
                prediction.loc[i, 'probability_'+str(count)] = np.nan        
        count += 1
    if feature_imp:
        return prediction , fi_data
    else:
        return prediction

In [ ]:
# Set the names here of proteins that fulfill a certain criteria 
# Here we use a list of proteins that that form liquid condensates in vitro and in vivo
uniprot_ids = '''Q99700
O60885
Q14781
P45973
P38432
Q7Z5Q1
O00571
Q9NQI0
O43781
Q04637
Q15056
P15502
Q01844
P22087
Q06787
P35637
Q13283
Q9UN86
P10071
P62993
Q13151
P09651
Q32P51
P22626
P51991
O14979
P31943
P55795
P31942
O43561
P10636
P43243
Q15648
P19338
P06748
Q15233
P52948
Q01860
P11940
P29590
Q8WXF1
Q96PK6
P98179
P23246
Q16637
P00441
Q07889
P23497
Q07955
Q01130
O95793
O75683
Q92804
Q13148
Q15554
P31483
Q01085
Q9UHD9
P46937
Q15059
P10644
P54727
Q13501
Q9NPI6
O00444
Q53HL2
Q9ULW0
Q9Y6A5
Q96LT7
P06748
Q16236
O43823
Q07157
Q9UDY2
O95049
Q9Y6M1
Q6NT89
Q8NC56
P04150
Q9BYJ9
Q9Y5A9
Q7Z739
P10997
Q9GZV5
P51608
O14979
P43351
Q08379
P08621
Q15424
Q16630
P18615
P48443
'''

# Input files, annotate and predict

In [75]:
# set path to file with sequences of interest to be predicted. 
# This file should be conform the fasta proteome format (e.g. something like below)
# >sp|Q96R72|OR4K3_HUMAN Olfactory receptor 4K3 OS=Homo sapiens OX=9606 GN=OR4K3 PE=3 SV=3
# FASTA SEQUENCE
# or
# >tr|QID2|name
# FASTA SEQUENCE
proteome_path= "/path/to/sequences.fa"

In [91]:
# In this step the input sequences of interest get annotated for amino acid content and related features. 
# Simultaneously, as .pkl file is generated and saved in the directory that has these features and can be loaded back in
# the notebooks (for example in the visualization notebook)
second_df = main(name='proteome_of_interest', fasta_path=proteome_path)

In [90]:
# save file with amino acid features as .csv if needed
# second_df.to_csv('path/of/interest/filename.csv')
# if needed this df can also be saved using
# compressed_pickle('output_name', second_df)
# second_df.head(10)

In [87]:
# now we annotate the llps proteins in the df if they were overlapping with the original training set. 
# This overlap is likely very low so you'll likely get a lot of messages saying that protein X and Y are not found. This 
# can be ignored. 
second_df = set_identifiers(second_df, uniprot_ids, identifier_name = 'llps')

In [92]:
# Do some final preprocessing on the data frame with sequences of interest
df2 = preprocess_and_scaledata(second_df, instance)

In [82]:
# load : get the data from file that was used to train the RF model
data = decompress_pickle('/path/to/RF_original_input_data.pbz2')

In [83]:
# Data filtering to make sure that very small (<100 AAs) and very big (> 3000 AAs) proteins are removed
# Just some other checks on the data that certain columns are not present
instance = 'llps'
data = data.loc[(data['length'] >= 100) & (data['length'] <= 3000)]
data = data.reset_index(drop=True)
domain_cols = data.columns[data.columns.str.contains("^domain")]
domain_data = data[domain_cols]
data.drop(columns=domain_cols)
data = preprocess_and_scaledata(data, instance)
label = 'llps'

(19504, 96)
Number of phase separating proteins in dataset: 90
(19504, 96)


In [93]:
# load pretrained RF model
clf = pickle.load(open('/path/to/finalized_RF.sav', 'rb'))

In [85]:
# now predict the sequences of interest. INPUT PARAMETERS:
# data = data (used to train the original classifier)
# clf = pretrained RF model
# second_df = df2 (which contains the sequences of interest)
prediction, fi_data = predict_proteome(data,
                                       clf,
                                       instance='llps',
                                       testing_size=.2,
                                       # remove_training=True,
                                       second_df=df2,
                                      )

100%|██████████| 216/216 [22:31<00:00,  6.26s/it]


In [18]:
# The prediction will contain 216 columns with prediction values (from 216 RF predictions). 
# Take the average of these values to get the PSAP score per sequence of interest

# save predictions to .csv file
# prediction.to_csv('path/of/interest/predicted_values.csv')

In [86]:
prediction.to_csv('predicted_values.csv', sep ='\t')